# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.linalg import pca
from cvx.risk.factor import FactorModel
from cvx.portfolio.min_risk import minrisk_problem

## Load prices and compute returns

In [2]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

## Compute principal components

In [3]:
factors = pca(returns=returns, n_components=10)

## Create the risk model, here a FactorModel

In [4]:
model = FactorModel(assets=len(returns.columns), k=10)

# update the model parameters
model.update(cov=factors.cov, exposure=factors.exposure.values, idiosyncratic_risk=factors.idiosyncratic.std().values,
             lower_assets=np.zeros(20), upper_assets=np.ones(20),
             lower_factors=-0.1*np.ones(10), upper_factors=0.1*np.ones(10))

# test the risk model with uniform weights
weights = 0.05 * np.ones(20)
model.estimate(weights).value

0.00923407730537884

## RiskModel is injected into optimizer

In [5]:
w = cvx.Variable(20)
y = cvx.Variable(10)

problem = minrisk_problem(model, w, y=y)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate(w, y=y).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)
print(y.value)

GOOG    2.359232e-09
AAPL    1.078173e-02
FB      2.289733e-09
BABA    6.149695e-02
AMZN    4.095655e-02
GE      1.692077e-03
AMD     9.033789e-10
WMT     4.448612e-02
BAC     1.048329e-09
GM      1.137605e-09
T       1.297920e-01
UAA     9.633963e-10
SHLD    7.309925e-10
XOM     2.502608e-01
RRC     1.020484e-09
BBY     1.392779e-02
MA      5.914576e-02
PFE     2.364381e-01
JPM     3.743304e-09
SBUX    1.510221e-01
dtype: float64
0.006799927138118909
[ 0.04251146 -0.09313427 -0.03753803  0.08220467  0.08363908 -0.02800705
 -0.1        -0.0717266   0.09784121 -0.03309495]


In [6]:
y.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])

In [7]:
model.parameter["exposure"].value @ w.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])

In [8]:
y.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])

In [9]:
model.parameter["exposure"].value @ w.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])

In [10]:
y.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])

In [11]:
model.parameter["exposure"].value @ w.value

array([ 0.04251146, -0.09313427, -0.03753803,  0.08220467,  0.08363908,
       -0.02800705, -0.1       , -0.0717266 ,  0.09784121, -0.03309495])